# Análisis Cuantitativo para Tesis: Obras Públicas en Puno (2013-2025)

**Autor de la Tesis:** Bach. Fabricio Rigoberto Rodriguez Ascarrunz
**Análisis de Datos por:** Ing. Jhon Wilber Ajata Ascarrunz

**Objetivo:** Este notebook reproduce el análisis cuantitativo descrito en el Capítulo IV de la tesis. Se parte de la base de datos **certificada** de INFOBRAS para generar las tablas de resultados y realizar las pruebas de hipótesis que comparan las modalidades de ejecución por Administración Directa y por Contrata.

## Metodología de Preparación de Datos

La base de datos utilizada en este análisis es el resultado de un riguroso proceso de limpieza, auditoría y curación realizado sobre el dataset crudo de INFOBRAS (`DataSet-Obras-Publicas-23-07-2025.xlsx`). Este proceso, documentado en su totalidad en el notebook `01_Data_Cleaning_INFOBRAS.ipynb`, aseguró la calidad y fiabilidad de los datos.

Los pasos clave del pre-procesamiento incluyeron:

1.  **Limpieza y Normalización de Texto:** Se eliminaron espacios en blanco y se normalizaron los caracteres (eliminación de acentos) en todas las columnas de texto para garantizar la consistencia de los filtros.
2.  **Estandarización de Columnas:** Todos los nombres de las columnas se convirtieron a formato `snake_case`.
3.  **Reparación Estructural:** Se renombraron columnas duplicadas para eliminar la ambigüedad (ej. `ruc.1` -> `ruc_supervisor`).
4.  **Transformación de Tipos de Datos:** Se convirtieron 16 columnas de fecha a `datetime` y 18 columnas numéricas a `float`, manejando errores de formato.
5.  **Auditoría y Curación de Datos:**
    *   Se identificaron y eliminaron **898** obras con fechas incoherentes (fecha de fin anterior a la de inicio).
    *   Se eliminaron **47,887** obras con plazos de ejecución no positivos (cero o negativos).
    *   Se eliminaron **19** obras con plazos extremos y poco plausibles (mayores a 10 años).
6.  **Manejo de Nulos:** Se aplicó una estrategia de imputación lógica y contextual para rellenar los valores nulos restantes.

El resultado de este proceso es el archivo `infobras_certificado_v1.csv`, que sirve como la fuente de datos certificada para el siguiente análisis.

In [1]:
# --- Paso 1: Configuración y Carga de Datos Certificados ---
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu

print("--- Cargando Dataset CERTIFICADO ---")
# Apuntamos a nuestro archivo de alta calidad generado en el notebook 01
df_certificado = pd.read_csv('../data/infobras_certificado_v1.csv', low_memory=False)

# Aseguramos que las columnas de fecha clave sean del tipo datetime
date_cols = ['fecha_de_inicio_de_obra', 'fecha_de_finalizacion_real']
for col in date_cols:
    df_certificado[col] = pd.to_datetime(df_certificado[col], errors='coerce')

# Filtramos para Puno
df_puno = df_certificado[df_certificado['departamento'] == 'PUNO'].copy()
print(f"Se han aislado {len(df_puno)} obras certificadas de Puno.")

--- Cargando Dataset CERTIFICADO ---
Se han aislado 8500 obras certificadas de Puno.


### 4.2 Análisis Comparativo del Cumplimiento Presupuestal

A continuación, se aísla la muestra de análisis para el primer objetivo específico y se ejecutan los cálculos y pruebas estadísticas correspondientes.

In [3]:
# --- Preparación de la muestra para análisis presupuestal ---
financiero_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_de_ejecucion_financiera_de_la_obra'] > 0)
].copy()

modalidades_de_interes = ['Administracion directa', 'Contrata']
financiero_comparison_sample = financiero_sample[financiero_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print("Composición de la muestra para análisis presupuestal:")
print(financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# --- Ejecución del Análisis Presupuestal ---
if not financiero_comparison_sample.empty and len(financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'].unique()) > 1:
    financiero_comparison_sample['desviacion_presupuestal_%'] = \
        ((financiero_comparison_sample['monto_de_ejecucion_financiera_de_la_obra'] - financiero_comparison_sample['monto_viable/aprobado']) 
         / financiero_comparison_sample['monto_viable/aprobado']) * 100

    admin_directa_dp = financiero_comparison_sample[financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion directa']['desviacion_presupuestal_%'].dropna()
    contrata_dp = financiero_comparison_sample[financiero_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()

    tabla_2_dp = pd.DataFrame({'Administracion directa': admin_directa_dp.describe(), 'Contrata': contrata_dp.describe()}).T[['count', 'mean', '50%', 'std', 'min', 'max']]
    tabla_2_dp.rename(columns={'50%': 'Mediana (%)', 'count': 'N', 'mean': 'Media (%)', 'std': 'Desv. Estándar', 'min': 'Mínimo', 'max': 'Máximo'}, inplace=True)
    tabla_2_dp['N'] = tabla_2_dp['N'].astype(int)
    
    print("\n--- Tabla 2: Estadísticas de Desviación Presupuestal (%) ---")
    display(tabla_2_dp.round(2))

    u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')
    print(f"\nResultado Prueba U de Mann-Whitney: p-value = {p_value_dp:.4f}")
else:
    print("\nNo hay suficientes datos en ambas modalidades para una comparación presupuestal directa.")

Composición de la muestra para análisis presupuestal:
modalidad_de_ejecucion_de_la_obra
Administracion directa    316
Contrata                   76
Name: count, dtype: int64

--- Tabla 2: Estadísticas de Desviación Presupuestal (%) ---


,N,Media (%),Mediana (%),Desv. Estándar,Mínimo,Máximo
Administracion directa,316,87.73,-16.63,1543.84,-99.98,25436.01
Contrata,76,-2.94,-9.87,129.41,-100.00,1010.66



Resultado Prueba U de Mann-Whitney: p-value = 0.2733


**Conclusión del Análisis Presupuestal:** Con un p-valor de 0.2733 (mayor a 0.05), no existe una diferencia estadísticamente significativa entre las medianas de desviación presupuestal. Sin embargo, el análisis descriptivo revela que la modalidad por Administración Directa presenta una variabilidad y un riesgo de sobrecostos extremos mucho mayor.

### 4.3 Análisis Comparativo de la Eficiencia en Plazos

Se prepara una nueva muestra independiente para evaluar el cumplimiento de los plazos, basándose únicamente en la disponibilidad de datos temporales.

In [6]:
# --- Preparación de la muestra para análisis de plazos ---
plazo_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['fecha_de_inicio_de_obra'].notna()) &
    (df_puno['fecha_de_finalizacion_real'].notna()) &
    (df_puno['plazo_de_ejecucion_en_dias'].notna())
].copy()

modalidades_de_interes = ['Administracion directa', 'Contrata']
plazo_comparison_sample = plazo_sample[plazo_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print("Composición de la muestra para análisis de plazos:")
print(plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

# --- Ejecución del Análisis de Plazos ---
if not plazo_comparison_sample.empty and len(plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'].unique()) > 1:
    plazo_comparison_sample['variacion_plazo_dias'] = \
        (plazo_comparison_sample['fecha_de_finalizacion_real'] - plazo_comparison_sample['fecha_de_inicio_de_obra']).dt.days - plazo_comparison_sample['plazo_de_ejecucion_en_dias']
        
    admin_directa_vp = plazo_comparison_sample[plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion directa']['variacion_plazo_dias'].dropna()
    contrata_vp = plazo_comparison_sample[plazo_comparison_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['variacion_plazo_dias'].dropna()
    
    tabla_3_vp = pd.DataFrame({'Administracion directa': admin_directa_vp.describe(), 'Contrata': contrata_vp.describe()}).T[['count', 'mean', '50%', 'std']]
    tabla_3_vp.rename(columns={'50%': 'Mediana (días)', 'count': 'N', 'mean': 'Media (días)', 'std': 'Desv. Estándar'}, inplace=True)
    tabla_3_vp['N'] = tabla_3_vp['N'].astype(int)

    print("\n--- Tabla 3: Estadísticas de Variación de Plazo (días) ---")
    display(tabla_3_vp.round(0))
    
    u_stat_vp, p_value_vp = mannwhitneyu(admin_directa_vp, contrata_vp, alternative='two-sided')
    print(f"\nResultado Prueba U de Mann-Whitney: p-value = {p_value_vp:.4f}")
else:
    print("\nNo hay suficientes datos en ambas modalidades para una comparación de plazos directa.")

Composición de la muestra para análisis de plazos:
modalidad_de_ejecucion_de_la_obra
Administracion directa    4587
Contrata                  1298
Name: count, dtype: int64

--- Tabla 3: Estadísticas de Variación de Plazo (días) ---


,N,Media (días),Mediana (días),Desv. Estándar
Administracion directa,4587,118.0,27.0,308.0
Contrata,1298,153.0,30.0,2048.0



Resultado Prueba U de Mann-Whitney: p-value = 0.5995


**Conclusión del Análisis de Plazos:** Con un p-valor de 0.5995 (mayor a 0.05), no existe una diferencia estadísticamente significativa en la mediana de los retrasos. Ambas modalidades presentan un retraso típico similar (menos de un mes). Sin embargo, la modalidad por Contrata muestra un riesgo mucho mayor de sufrir retrasos extremos.

### 4.4 Análisis Comparativo de Mecanismos de Control

Finalmente, se analiza la población total de obras en Puno para comparar los indicadores de control que no dependen del estado de finalización.

In [7]:
# --- Preparación y Ejecución del Análisis de Control ---
df_puno['fue_paralizado'] = (df_puno['causal_de_paralizacion'] != 'No Paralizada').astype(int)
control_comparison_sample = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print("Composición de la muestra para análisis de control (Población Total):")
print(control_comparison_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

control_analysis = control_comparison_sample.groupby('modalidad_de_ejecucion_de_la_obra').agg(
    total_obras=('codigo_infobras', 'count'),
    promedio_informes_control=('n_informes_de_control', 'mean'),
    total_con_controversias=('n_de_controversias', lambda x: (x > 0).sum()),
    total_paralizadas=('fue_paralizado', 'sum')
)
control_analysis['Obras con Controversias (%)'] = (control_analysis['total_con_controversias'] / control_analysis['total_obras']) * 100
control_analysis['Tasa de Paralizacion (%)'] = (control_analysis['total_paralizadas'] / control_analysis['total_obras']) * 100
tabla_4_control = control_analysis[['total_obras', 'promedio_informes_control', 'Obras con Controversias (%)', 'Tasa de Paralizacion (%)']]

print("\n--- Tabla 4: Indicadores de Transparencia y Control ---")
display(tabla_4_control.round(2))

Composición de la muestra para análisis de control (Población Total):
modalidad_de_ejecucion_de_la_obra
Administracion directa    6491
Contrata                  1625
Name: count, dtype: int64

--- Tabla 4: Indicadores de Transparencia y Control ---


,total_obras,promedio_informes_control,Obras con Controversias (%),Tasa de Paralizacion (%)
modalidad_de_ejecucion_de_la_obra,,,,
Administracion directa,6491,0.35,0.03,6.32
Contrata,1625,0.70,3.02,6.34


**Conclusión del Análisis de Control:** La modalidad por Contrata es un sistema mucho más formalizado, generando el doble de informes de control y 100 veces más controversias registradas. Sin embargo, el riesgo de que una obra se paralice es virtualmente idéntico en ambas modalidades, sugiriendo que es un riesgo sistémico de la región.